In [1]:
upstream = ['01-get']
product = None

In [2]:
# Parameters
upstream = {"01-get": {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/products/extract-pipeline.ipynb", "csv_file": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/data/house_prices.csv"}}
product = {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/products/cleanup.ipynb", "clean_csv": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/data/house_prices_clean.csv"}


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import re
import pandas as pd

In [5]:
df = pd.read_csv(upstream['01-get']['csv_file'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29451 non-null  object 
 1   UNDER_CONSTRUCTION     29451 non-null  int64  
 2   RERA                   29451 non-null  int64  
 3   BHK_NO.                29451 non-null  int64  
 4   BHK_OR_RK              29451 non-null  object 
 5   SQUARE_FT              29451 non-null  float64
 6   READY_TO_MOVE          29451 non-null  int64  
 7   RESALE                 29451 non-null  int64  
 8   ADDRESS                29451 non-null  object 
 9   LONGITUDE              29451 non-null  float64
 10  LATITUDE               29451 non-null  float64
 11  TARGET(PRICE_IN_LACS)  29451 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ MB


In [7]:
df.isna().sum()

POSTED_BY                0
UNDER_CONSTRUCTION       0
RERA                     0
BHK_NO.                  0
BHK_OR_RK                0
SQUARE_FT                0
READY_TO_MOVE            0
RESALE                   0
ADDRESS                  0
LONGITUDE                0
LATITUDE                 0
TARGET(PRICE_IN_LACS)    0
dtype: int64

In [8]:
# Rename columns to snake case
renamed_mapping = {}
for c in df.columns:
    new_c_name = re.sub('[^0-9a-zA-Z_]+', '', c)
    renamed_mapping[c] = new_c_name.lower()  # .replace(' ', '_').replace('(', '').replace(')', '')
renamed_mapping['targetprice_in_lacs'] = 'target_price_in_lacs'

df = df.rename(columns=renamed_mapping)

In [9]:
# Convert price to dolllars 1 lac == 1,397
df['price'] = df['targetprice_in_lacs'] * 1_397

# Convert square_ft to square meters
df['area_m2'] = df['square_ft'] * 0.092903

In [10]:
df = df.drop(columns=['square_ft', 'targetprice_in_lacs'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   posted_by           29451 non-null  object 
 1   under_construction  29451 non-null  int64  
 2   rera                29451 non-null  int64  
 3   bhk_no              29451 non-null  int64  
 4   bhk_or_rk           29451 non-null  object 
 5   ready_to_move       29451 non-null  int64  
 6   resale              29451 non-null  int64  
 7   address             29451 non-null  object 
 8   longitude           29451 non-null  float64
 9   latitude            29451 non-null  float64
 10  price               29451 non-null  float64
 11  area_m2             29451 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ MB


In [11]:
# Extracting cities from address
def get_city(value):
    parts = value.split(',')
    return parts[-1:][0]

# Too many cities, heatmap not working.
# df['city'] = df['address'].apply(get_city)

# df['city'].value_counts()

In [12]:
df = df.drop(columns=['address'])

In [13]:
df['posted_by'].value_counts()

posted_by
Dealer     18291
Owner      10538
Builder      622
Name: count, dtype: int64

In [14]:
df['bhk_or_rk'].value_counts()

bhk_or_rk
BHK    29427
RK        24
Name: count, dtype: int64

In [15]:
# df = pd.get_dummies(df, columns=['posted_by', 'bhk_or_rk', 'city'])
df = pd.get_dummies(df, columns=['posted_by', 'bhk_or_rk'])

In [16]:
df.head()

,under_construction,rera,bhk_no,ready_to_move,resale,longitude,latitude,price,area_m2,posted_by_Builder,posted_by_Dealer,posted_by_Owner,bhk_or_rk_BHK,bhk_or_rk_RK
0,0,0,2,1,1,12.969910,77.597960,76835.0,120.795863,False,False,True,True,False
1,0,0,2,1,1,12.274538,76.644605,71247.0,118.451325,False,True,False,True,False
2,0,0,2,1,1,12.778033,77.632191,60071.0,86.693338,False,False,True,True,False
3,0,1,2,1,1,28.642300,77.344500,87312.5,86.392464,False,False,True,True,False
4,1,0,2,0,1,22.592200,88.484911,84518.5,92.810956,False,True,False,True,False


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   under_construction  29451 non-null  int64  
 1   rera                29451 non-null  int64  
 2   bhk_no              29451 non-null  int64  
 3   ready_to_move       29451 non-null  int64  
 4   resale              29451 non-null  int64  
 5   longitude           29451 non-null  float64
 6   latitude            29451 non-null  float64
 7   price               29451 non-null  float64
 8   area_m2             29451 non-null  float64
 9   posted_by_Builder   29451 non-null  bool   
 10  posted_by_Dealer    29451 non-null  bool   
 11  posted_by_Owner     29451 non-null  bool   
 12  bhk_or_rk_BHK       29451 non-null  bool   
 13  bhk_or_rk_RK        29451 non-null  bool   
dtypes: bool(5), float64(4), int64(5)
memory usage: 2.2 MB


In [18]:
for c in df.columns:
    if df[c].dtype == bool:
        print(f'{c} {df[c].dtype}')
        df[c] = df[c].astype(int)

df.info()

posted_by_Builder bool
posted_by_Dealer bool
posted_by_Owner bool
bhk_or_rk_BHK bool
bhk_or_rk_RK bool
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   under_construction  29451 non-null  int64  
 1   rera                29451 non-null  int64  
 2   bhk_no              29451 non-null  int64  
 3   ready_to_move       29451 non-null  int64  
 4   resale              29451 non-null  int64  
 5   longitude           29451 non-null  float64
 6   latitude            29451 non-null  float64
 7   price               29451 non-null  float64
 8   area_m2             29451 non-null  float64
 9   posted_by_Builder   29451 non-null  int64  
 10  posted_by_Dealer    29451 non-null  int64  
 11  posted_by_Owner     29451 non-null  int64  
 12  bhk_or_rk_BHK       29451 non-null  int64  
 13  bhk_or_rk_RK        29451 non-null  int64  
dtyp

In [19]:
# 1. hot shoe encoding
# 2. Drop encoded columns
# 3. Extract city
# 4.

In [20]:

df.to_csv(product['clean_csv'], index=False)